In [7]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_80fdb1cbcfba4ffc89754c5419936eaf = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-KpsSgZ6YTByw0VvYt62SM6470OAy9_yGH0B4gTdKFzL',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_80fdb1cbcfba4ffc89754c5419936eaf.get_object(Bucket='catsanddogs-donotdelete-pr-uve7xoh7zeegdm',Key='Book1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)



In [6]:
df_data_1.head()

,Postcode,Borough,Neighbourhood
0,M9Z,Not assigned,Not assigned
1,M9Y,Not assigned,Not assigned
2,M9X,Not assigned,Not assigned
3,M9W,Etobicoke,Northwest
4,M9V,Etobicoke,Albion Gardens
